# 피마인디언 당뇨병 예측

In [3]:
import numpy as np
import pandas as pd

- 데이터 전처리

In [4]:
# csv 데이터의 1~9번째 줄까지 필요없는 데이터 -> skiprows=9 활용해서 이후 데이터 불러옴
# 일반적으로 제일 윗줄을 header로 인식하므로 header=None -> 10번째 줄부터 데이터로 추출
df = pd.read_csv('pima-indians-diabetes.csv', skiprows=9, header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


- 컬럼명 지정

In [5]:
# 1. Number of times pregnant
# 2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
# 3. Diastolic blood pressure (mm Hg)
# 4. Triceps skin fold thickness (mm)
# 5. 2-Hour serum insulin (mu U/ml)
# 6. Body mass index (weight in kg/(height in m)^2)
# 7. Diabetes pedigree function
# 8. Age (years)
# 9. Class variable (0 or 1)
df.columns = ['P', 'G', 'BP', 'S', 'I', 'BMI', 'D', 'Age', 'Class']
df.head()

,P,G,BP,S,I,BMI,D,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


- Class 기준으로 전반부 X 데이터 / 후반부 y 데이터 < ☆중요☆ >

In [7]:
# .iloc[] -> 넘파이 indexing 사용 가능
# 데이터 프레임 상태로 바로 활용 가능
X = df.iloc[:, :-1]
X.head()

,P,G,BP,S,I,BMI,D,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [17]:
# 결측치 데이터가 있는지 확인 必
# 0 이면 결측치 없는 것
# .isna() 또는 .isnull()
df.isna().sum()

P        0
G        0
BP       0
S        0
I        0
BMI      0
D        0
Age      0
Class    0
dtype: int64

In [53]:
# .values - 데이터 프레임의 값(데이터)만 추출
X = df.iloc[:, :-1].values
type(X) 

numpy.ndarray

In [10]:
# .values 희생 -> 실수로 모두 바뀜
# 왜 바뀌었을까? numpy array가 되면 더 많은 데이터 표기를 위해 데이터 모두 실수로 바뀜
# X[:5]

In [29]:
# 여러가지 방법으로 y 데이터 추출
y = df.Class
y = df['Class'].values
y = df.Class.values
y = df.iloc[:, -1].values

In [5]:
X.shape, y.shape

((768, 8), (768,))

- Train/Test dataset 으로 분리

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2021
)

In [32]:
# y 값 분포 확인
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([500, 268], dtype=int64))

In [33]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([400, 214], dtype=int64))

- GridSearchCV, DecisionTreeClassifier

In [34]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2021)
params = {
    'max_depth' : [2,3,4,5,6],
    'min_samples_split' : [2,3,4]
}

- max_depth : 트리를 만들 때 노드의 깊이 ; 가지치기 층수 
- min_samples_split : 데이터 분리의 최소값

=> (max, min) : (2, 2), (2, 3), (2, 4), (3,2) .... 

In [35]:
from sklearn.model_selection import GridSearchCV
grid_dt = GridSearchCV(dtc, param_grid=params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [38]:
# 가장 성능이 좋았던 params 조합
# estimator의 결과값 끝에 _가 붙어있음
grid_dt.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [39]:
grid_dt.best_estimator_.score(X_test, y_test)

0.7337662337662337

In [49]:
best_dt = grid_dt.best_estimator_
best_dt.score(X_test, y_test)

0.7337662337662337

- 실제 적용

In [43]:
X_test[10], y_test[10]

(array([ 12.   , 100.   ,  84.   ,  33.   , 105.   ,  30.   ,   0.488,
         46.   ]),
 0)

In [46]:
# .predict() : [ [], [], ... [] ] 여러 개의 데이터 예측하게 만들어짐
# 그래서 [X_test[10]] or X_test[10].reshape(1,-1) 입력해서 2차원으로 만듦
test_data = X_test[10].reshape(1,-1)
pred = best_dt.predict(test_data)
pred

array([0], dtype=int64)

In [51]:
print('양성' if pred[0] == 1 else '음성') # 3항 연산자

음성


In [50]:
# if pred[0] == 1:
#     print('양성')
# else:
#     print('음성')

- 참고 사항

In [25]:
df2 = pd.DataFrame(np.arange(25).reshape(5,5), index=list('abcde'), columns=list('vwxyz'))
df2

,v,w,x,y,z
a,0,1,2,3,4
b,5,6,7,8,9
c,10,11,12,13,14
d,15,16,17,18,19
e,20,21,22,23,24


In [27]:
# np.nan으로 바뀌면서 컬럼이 실수 형태로 바뀜
# 데이터 프레임 컬럼 단위로 동일한 형태의 데이터여야만 함
df2['v']['c'] = np.nan
df2.z['d'] = np.nan
df2

<ipython-input-27-c6acd0e09a21>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['v']['c'] = np.nan
<ipython-input-27-c6acd0e09a21>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.z['d'] = np.nan


,v,w,x,y,z
a,0.0,1,2,3,4.0
b,5.0,6,7,8,9.0
c,NaN,11,12,13,14.0
d,15.0,16,17,18,NaN
e,20.0,21,22,23,24.0


In [23]:
# 데이터 프레임의 컬럼별 결측치 확인
df2.isna().sum()

v    1
w    0
x    0
y    0
z    1
dtype: int64

In [24]:
# 전체 데이터 프레임의 결측치 확인
df2.isna().sum().sum()

2

#### ※ X, y 데이터를 데이터 프레임 / NP array 로 사용할 때 다른점 
- 데이터 프레임 : 데이터와 인덱스 함께 움직임 (원본 그대로)
- NP array : 바뀐 데이터에 맞춰 인덱스 자동으로 변환됨
- 슬라이싱
    - 데이터 프레임 : X_train.iloc[0]
    - NP array : X_train[0]

-> 즉, 나중을 위해서 NP array 로 변환해서 사용하길 바란다.  